# Mesh Visualization with PyRender

This notebook loads and visualizes 3D meshes from the LOD dataset using PyRender and matplotlib for display.

In [ ]:
import os
import trimesh
import pyrender
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import imageio
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, Dropdown

## Configuration

In [ ]:
# Configuration
dataset_root = "/home/jeans/progressive_img2sketch/rtsc-1.6/LOD_data_50"
output_root = "/home/jeans/progressive_img2sketch/rtsc-1.6/model_images"

# Create output root directory if it doesn't exist
os.makedirs(output_root, exist_ok=True)

# List available scenes
available_scenes = []
for scene_folder_name in sorted(os.listdir(dataset_root)):
    scene_folder_path = os.path.join(dataset_root, scene_folder_name)
    if os.path.isdir(scene_folder_path) and scene_folder_name.isdigit():
        available_scenes.append(scene_folder_name)

print(f"Available scenes: {available_scenes}")
print(f"Total scenes: {len(available_scenes)}")

## Helper Functions

In [ ]:
def load_mesh(mesh_path):
    """Load a mesh from file path and handle Scene objects."""
    try:
        mesh = trimesh.load(mesh_path)
        
        # Handle Scene objects (multiple meshes)
        if isinstance(mesh, trimesh.scene.Scene):
            combined_meshes = []
            for geo_name, geo_obj in mesh.geometry.items():
                if isinstance(geo_obj, trimesh.Trimesh):
                    combined_meshes.append(geo_obj)
                elif isinstance(geo_obj, trimesh.scene.Scene):
                    # Handle nested scenes
                    for sub_geo_name, sub_geo_obj in geo_obj.geometry.items():
                        if isinstance(sub_geo_obj, trimesh.Trimesh):
                            combined_meshes.append(sub_geo_obj)
            
            if combined_meshes:
                mesh = trimesh.util.concatenate(tuple(combined_meshes))
            else:
                return None
        
        # Validate mesh
        if mesh.is_empty or mesh.vertices.shape[0] == 0 or mesh.faces.shape[0] == 0:
            return None
            
        return mesh
        
    except Exception as e:
        print(f"Error loading mesh {mesh_path}: {e}")
        return None

def setup_camera(mesh, angle_azimuth=0, angle_elevation=15):
    """Setup camera for viewing the mesh."""
    # Calculate bounding box of the mesh
    bounds = mesh.bounds
    center = mesh.centroid
    
    # Determine a suitable radius for the camera
    max_extent = np.max(np.linalg.norm(bounds - center, axis=1))
    view_distance_multiplier = 2.5
    radius = max_extent * view_distance_multiplier
    
    # Convert angles to radians
    azimuth_rad = np.radians(angle_azimuth)
    elevation_rad = np.radians(angle_elevation)
    
    # Calculate camera position
    x = radius * np.cos(elevation_rad) * np.sin(azimuth_rad)
    y = radius * np.sin(elevation_rad)
    z = radius * np.cos(elevation_rad) * np.cos(azimuth_rad)
    
    camera_position = center + np.array([x, y, z])
    
    # Construct view matrix
    forward = center - camera_position
    forward = forward / np.linalg.norm(forward)
    
    up_vec = np.array([0.0, 1.0, 0.0])
    right = np.cross(forward, up_vec)
    
    if np.linalg.norm(right) < 1e-6:
        right = np.array([1.0, 0.0, 0.0]) if forward[1] > 0 else np.array([-1.0, 0.0, 0.0])
    
    right = right / np.linalg.norm(right)
    up = np.cross(right, forward)
    up = up / np.linalg.norm(up)
    
    # Create view matrix
    view_matrix = np.array([
        [right[0], right[1], right[2], -np.dot(right, camera_position)],
        [up[0], up[1], up[2], -np.dot(up, camera_position)],
        [-forward[0], -forward[1], -forward[2], np.dot(forward, camera_position)],
        [0.0, 0.0, 0.0, 1.0]
    ])
    
    camera_pose = np.linalg.inv(view_matrix)
    
    # Create camera
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
    camera_node = pyrender.Node(camera=camera, matrix=camera_pose)
    
    return camera_node

def render_mesh(mesh, angle_azimuth=0, angle_elevation=15, width=640, height=480):
    """Render a mesh with pyrender."""
    # Create scene
    scene = pyrender.Scene(bg_color=[0.0, 0.0, 0.0, 0.0])
    
    # Add mesh
    mesh_node = pyrender.Mesh.from_trimesh(mesh)
    scene.add(mesh_node)
    
    # Add lighting
    light_color = np.array([1.0, 1.0, 1.0])
    
    # Key light
    key_light_pose = np.eye(4)
    key_light_pose[:3, 3] = np.array([15.0, 15.0, 10.0])
    scene.add(
        pyrender.DirectionalLight(color=light_color, intensity=20.0),
        pose=key_light_pose
    )
    
    # Fill light
    fill_light_pose = np.eye(4)
    fill_light_pose[:3, 3] = np.array([-15.0, 10.0, 5.0])
    scene.add(
        pyrender.DirectionalLight(color=light_color, intensity=8.0),
        pose=fill_light_pose
    )
    
    # Rim light
    rim_light_pose = np.eye(4)
    rim_light_pose[:3, 3] = np.array([0.0, 10.0, -20.0])
    scene.add(
        pyrender.DirectionalLight(color=light_color, intensity=12.0),
        pose=rim_light_pose
    )
    
    # Add camera
    camera_node = setup_camera(mesh, angle_azimuth, angle_elevation)
    scene.add_node(camera_node)
    
    # Render
    r = pyrender.OffscreenRenderer(viewport_width=width, viewport_height=height)
    color, depth = r.render(scene)
    r.delete()
    
    return color, depth

def get_mesh_info(mesh):
    """Get information about a mesh."""
    info = {
        'vertices': mesh.vertices.shape[0],
        'faces': mesh.faces.shape[0],
        'bounds': mesh.bounds,
        'centroid': mesh.centroid,
        'volume': mesh.volume if mesh.is_watertight else "N/A (not watertight)",
        'surface_area': mesh.area,
        'is_watertight': mesh.is_watertight
    }
    return info

## Load and Visualize a Single Scene

In [ ]:
# Select a scene to visualize
scene_id = available_scenes[0] if available_scenes else "0"
print(f"Loading scene: {scene_id}")

scene_folder_path = os.path.join(dataset_root, scene_id)
lod_meshes = {
    "lod1": os.path.join(scene_folder_path, "lod1.obj"),
    "lod2": os.path.join(scene_folder_path, "lod2.obj"),
    "lod3": os.path.join(scene_folder_path, "lod3.obj"),
    # "lod4": os.path.join(scene_folder_path, "lod4.obj"),
}

# Load all available LOD meshes
loaded_meshes = {}
for lod_name, lod_path in lod_meshes.items():
    if os.path.exists(lod_path):
        mesh = load_mesh(lod_path)
        if mesh is not None:
            loaded_meshes[lod_name] = mesh
            print(f"Loaded {lod_name}: {lod_path}")
        else:
            print(f"Failed to load {lod_name}: {lod_path}")
    else:
        print(f"File not found: {lod_path}")

print(f"\nLoaded {len(loaded_meshes)} meshes: {list(loaded_meshes.keys())}")

## Mesh Information

In [ ]:
# Display mesh information
for lod_name, mesh in loaded_meshes.items():
    info = get_mesh_info(mesh)
    print(f"\n{lod_name.upper()} Information:")
    print(f"  Vertices: {info['vertices']:,}")
    print(f"  Faces: {info['faces']:,}")
    print(f"  Bounds: {info['bounds']}")
    print(f"  Centroid: {info['centroid']}")
    print(f"  Volume: {info['volume']}")
    print(f"  Surface Area: {info['surface_area']:.2f}")
    print(f"  Is Watertight: {info['is_watertight']}")

## Static Visualization - All LODs

In [ ]:
# Render all LODs for comparison
if loaded_meshes:
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, (lod_name, mesh) in enumerate(loaded_meshes.items()):
        if i < 4:  # Only show first 4 LODs
            color, depth = render_mesh(mesh, angle_azimuth=45, angle_elevation=15)
            axes[i].imshow(color)
            axes[i].set_title(f"{lod_name.upper()} - {get_mesh_info(mesh)['vertices']:,} vertices")
            axes[i].axis('off')
    
    # Hide unused subplots
    for j in range(len(loaded_meshes), 4):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No meshes loaded for visualization")

## Interactive Visualization

In [ ]:
# Interactive visualization with sliders
if loaded_meshes:
    def interactive_render(lod_name, azimuth, elevation):
        if lod_name in loaded_meshes:
            mesh = loaded_meshes[lod_name]
            color, depth = render_mesh(mesh, angle_azimuth=azimuth, angle_elevation=elevation)
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Color render
            ax1.imshow(color)
            ax1.set_title(f"{lod_name.upper()} - Color Render\nAzimuth: {azimuth}°, Elevation: {elevation}°")
            ax1.axis('off')
            
            # Depth render
            im = ax2.imshow(depth, cmap='viridis')
            ax2.set_title(f"{lod_name.upper()} - Depth Map")
            ax2.axis('off')
            plt.colorbar(im, ax=ax2, shrink=0.8)
            
            plt.tight_layout()
            plt.show()
            
            # Show mesh info
            info = get_mesh_info(mesh)
            print(f"Mesh Info - Vertices: {info['vertices']:,}, Faces: {info['faces']:,}")
    
    # Create interactive widget
    lod_dropdown = Dropdown(
        options=list(loaded_meshes.keys()),
        value=list(loaded_meshes.keys())[0],
        description='LOD:'
    )
    
    azimuth_slider = IntSlider(
        value=45,
        min=0,
        max=360,
        step=15,
        description='Azimuth:'
    )
    
    elevation_slider = IntSlider(
        value=15,
        min=-30,
        max=60,
        step=15,
        description='Elevation:'
    )
    
    interact(interactive_render, 
             lod_name=lod_dropdown, 
             azimuth=azimuth_slider, 
             elevation=elevation_slider)
    
else:
    print("No meshes loaded for interactive visualization")

## Multiple Camera Angles - Grid View

In [ ]:
# Render multiple camera angles for a single LOD
if loaded_meshes:
    # Select first available LOD
    selected_lod = list(loaded_meshes.keys())[0]
    selected_mesh = loaded_meshes[selected_lod]
    
    # Camera angles
    azimuth_angles = [0, 45, 90, 135, 180, 225, 270, 315]
    elevation_angles = [0, 15, 30]
    
    # Create a grid of renders
    fig, axes = plt.subplots(len(elevation_angles), len(azimuth_angles), 
                            figsize=(20, 8))
    
    for i, elevation in enumerate(elevation_angles):
        for j, azimuth in enumerate(azimuth_angles):
            color, depth = render_mesh(selected_mesh, 
                                     angle_azimuth=azimuth, 
                                     angle_elevation=elevation,
                                     width=320, height=240)
            
            ax = axes[i, j] if len(elevation_angles) > 1 else axes[j]
            ax.imshow(color)
            ax.set_title(f"A:{azimuth}° E:{elevation}°", fontsize=8)
            ax.axis('off')
    
    plt.suptitle(f"{selected_lod.upper()} - Multiple Camera Angles", fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No meshes loaded for grid visualization")

## Browse Different Scenes

In [ ]:
# Interactive scene browser
def browse_scenes(scene_id, lod_name):
    scene_folder_path = os.path.join(dataset_root, scene_id)
    lod_path = os.path.join(scene_folder_path, f"{lod_name}.obj")
    
    if os.path.exists(lod_path):
        mesh = load_mesh(lod_path)
        if mesh is not None:
            # Render from multiple angles
            angles = [0, 90, 180, 270]
            fig, axes = plt.subplots(1, 4, figsize=(16, 4))
            
            for i, angle in enumerate(angles):
                color, depth = render_mesh(mesh, angle_azimuth=angle, angle_elevation=15)
                axes[i].imshow(color)
                axes[i].set_title(f"Angle: {angle}°")
                axes[i].axis('off')
            
            plt.suptitle(f"Scene {scene_id} - {lod_name.upper()}")
            plt.tight_layout()
            plt.show()
            
            # Show mesh info
            info = get_mesh_info(mesh)
            print(f"Scene {scene_id} - {lod_name.upper()}:")
            print(f"  Vertices: {info['vertices']:,}, Faces: {info['faces']:,}")
            print(f"  Volume: {info['volume']}, Surface Area: {info['surface_area']:.2f}")
        else:
            print(f"Failed to load mesh: {lod_path}")
    else:
        print(f"File not found: {lod_path}")

if available_scenes:
    scene_dropdown = Dropdown(
        options=available_scenes[:10],  # Show first 10 scenes
        value=available_scenes[0],
        description='Scene:'
    )
    
    lod_dropdown = Dropdown(
        options=['lod1', 'lod2', 'lod3', 'lod4'],
        value='lod1',
        description='LOD:'
    )
    
    interact(browse_scenes, scene_id=scene_dropdown, lod_name=lod_dropdown)
else:
    print("No scenes available for browsing")